# Chain level Steenrod operations

## Introduction <a name="introduction"></a>

After Steenrod's seminal work [[1]](#1) introducing power operations in the mod-2 cohomology of spaces, analogue structures have been constructed in a wide range of context including, for any prime $p$, the mod-$p$ cohomology of spaces [[2, 3]](#23) and mod-$p$ homology of loop spaces [[4, 5]](#45). A unified algebraic framework for the study of Steenrod operations was given by May in [[6]](#6) and the introduction of operads [[7]](#7) brought conceptual clarity to the subject. More specifically, the operads relevant to the study of Steenrod operations are $E_\infty$-operads, the algebras of which are commutative up to coherent homotopies.

In the literature there exist two algebraic models for the $E_\infty$-operad whose internal combinatorics is suited for effective constructions, these are the Barratt-Eccles and Surjection operads [[8, 9]](#89). Using our implementation of these operads, in this notebook we construct elements in each of them representing Steenrod operations on the (co)homology of their algebras. For the cochains of spaces considered as an $E_\infty$-algebra, the action of these Steenrod operation elements generalize the cup-$i$ products and we illustrate it using our implementation of the Eilenberg-Zilber operad, i.e., the endomorphism operad of the functor of normalized chains.


### About the format

The interactive format used in this work is a [Jupyter Notebook](https://jupyter.readthedocs.io/en/latest/index.html). It contains text cells, like this one, and executable cells where [Python](https://www.python.org/) code can run. For the reader unfamiliar with this format we recommend clicking *Cell* and from the dropdown menu choose *Run All* after an overview of this document.

# Table of contents
1. [Introduction](#introduction)
2. [Preliminaries](#preliminaries)
    1. [Free modules](#free_modules)
    2. [Symmetric groups](#symmetric_group)
    3. [Symmetric rings](#symmetric_rings)
3. [Operads](#operads)
    1. [Barratt-Eccles](#barrat_eccles)
    2. [Surjection](#surjection)
4. [Steenrod operations](#steenrod_operations)
    1. [Chain convention](#chain_convention)
    2. [Cochain convention](#cochain_convention)
5. [Cochains of spaces](#spaces)
    1. [Simplicial](#simplicial)
    2. [Cubical](#cubical)
6. [References](#references)

## Preliminaries <a name="preliminaries"></a>

### Free modules <a name="free_modules"></a>

**Definition**. Let $R$ be a ring and $B$ a set. The free $R$-module generated by $B$ consists of all $R$-linear combination of elements in $B$

\begin{equation*}
R[B] = \Big\{ \sum_i r_ib_i\ |\ r_i \in R, b_i \in B \Big\}.
\end{equation*}

**Implementation**. For a ring $R$ equal to $\mathbb Z$ or $\mathbb Z/m\mathbb Z$ we introduce a class modeling elements in free $R$-modules. The ring is specified via the class attribute `torsion`, where `'free'` sets $R = \mathbb Z$ and a positive `int` sets $R = \mathbb Z/m\mathbb Z$. The class supports `+` addition, `-` substraction and `*` scaling.

In [1]:
from clesto import Module_element

# initialization: a dict with int values
x = Module_element({'a':1, 'b':2, 'c':3})

# arithmetic operations
print(f'  x = {x} \n -x = {-x} \nx+x = {x+x} \n2*x = {2*x} \nx-x = {x-x}')

  x = a + 2b + 3c 
 -x = - a - 2b - 3c 
x+x = 2a + 4b + 6c 
2*x = 4a + 8b + 12c 
x-x = 0


In [2]:
# The attribute torsion can be set in three ways:
# a) For all future instances using the class attribute default_torsion.
# b) Using a flag at initialization of an instance.
# c) Using a setter method on an initialized instance.

Module_element.default_torsion = 'free'
x = Module_element({'a':1, 'b':2, 'c':3})
print(f'x = {x} with torsion = {x.torsion}')
y = Module_element(x, torsion=3)
print(f'x = {y} with torsion = {y.torsion}')
x.set_torsion(2)
print(f'x = {x} with torsion = {x.torsion}')

x = a + 2b + 3c with torsion = free
x = a + 2b with torsion = 3
x = a + c with torsion = 2


### Symmetric groups <a name="symmetric_groups"></a>

**Definition**. Let $\mathrm S_r$ denote the group of permutations of $r$ elements. That is to say, the set of bijections from $\{1,\dots,r\}$ to itself. A permutation $\pi$ can be represented by the tuple of its images $(\pi(1), \dots, \pi(r))$ and we refer to $r$ as the arity of $\pi$.

**Implementation**. We introduce a subclass of `tuple` modeling elements in $\mathrm S_r$ supporting the group multiplication `*`.

In [3]:
from clesto import SymmetricGroup_element

x = SymmetricGroup_element((2,3,1))
print(f'x = {x} \nx*x = {x*x} \nx*x*x = {x*x*x}')

x = (2,3,1) 
x*x = (3,1,2) 
x*x*x = (1,2,3)


### Symmetric rings <a name="symmetric_groups"></a>

**Definition**. Let $R$ be a ring and $\Gamma$ a group. The free $R$-module generated by $\Gamma$ is a ring with product

\begin{equation*}
\left( \sum_i r_ia_i \right) \left( \sum_j s_jb_j \right) = \sum_{i,j} (r_is_j)(a_ib_{j}).
\end{equation*}

**Implementation**. We introduce a subclass of `Module_element` that models elements in the group rings $\mathbb Z[\mathrm S_r]$ or $\mathbb Z/n\mathbb Z[\mathrm S_r]$. It inherents addition `+`, substraction `-`, scaling `*` by integers and implements the group ring product `*`.

In [11]:
from clesto import SymmetricRing_element

x = SymmetricRing_element({(2,3,1):-1, (1,3,2):1})
print(f'If x = {x} then \nx*x = {x*x} and \nx*x*x = {x*x*x}')

If x = - (2,3,1) + (1,3,2) then 
x*x = (3,1,2) - (2,1,3) - (3,2,1) + (1,2,3) and 
x*x*x = - (1,2,3) + (3,2,1) + 2(1,3,2) - 2(2,3,1) + (2,1,3) - (3,1,2)


#### Composition of symmetric module elements

We will describe a construction that given $\pi \in \Sigma_r$ and $\tau_j \in \Sigma_{s_j}$ for $j = \{1, \dots, r\}$ produces a permutation in $\Sigma_{s_1 + \cdots + s_k - 1}$. We begin by partition $\{1, \dots, s_1 + \cdots + s_k - 1\}$ into $r$ consecutive subsets, apply $s_j$ to the $j$-th one and $\pi$ to the subsets themselves. Precisely, we have the following

**Definition**. For integers $r, s \geq 1$ and $k \in \{1, \ldots, s\}$ the partial composition is the linear map 

\begin{equation*}
\circ_k : R[\Sigma_r] \otimes R[\Sigma_s] \to R[\Sigma_{r+s-1}]
\end{equation*}
is defined on basis elements by
\begin{equation*}
(\pi \circ_k \tau)(i) = \begin{cases}
Finish \\ Please.
\end{cases}
\end{equation*}
The total composition is the linear map
\begin{equation*}
	\circ_{\Sigma} : R[\Sigma_r] \otimes R[\Sigma_{s_1}] \otimes \cdots \otimes R[\Sigma_{s_r}] \ \to \
	R[\Sigma_{s_1 + \cdots + s_{r}}] \, .
\end{equation*}

is defined recursively by
\begin{equation*}
A\ formula\ please.
\end{equation*}

**Implementation**. The `Symmetric_Module_element` class has the method `compose` which given either a collection of`Symmetric_Module_element` instances or a single instance and an `int` provides either the total or partial composition with `self`.

In [ ]:
from clesto import SymmetricModule_element

# partial composition
x = SymmetricModule_element({(3,2,1):1})
y = SymmetricModule_element({(1,3,2):1, (1,2,3):2})
print(f'If x = {x} and y = {y}\nthen x o_2 y = {x.compose(y,2)}\n')

# total composition
x = SymmetricModule_element({(3,2,1):1, (1,2,3):2})
y1 = SymmetricModule_element({(1,2):1})
y2 = SymmetricModule_element({(1,3,4,5,2):1})
y3 = SymmetricModule_element({(4,1,3,2):1})
print(f'x = {x}\ny1 = {y1}\ny2 = {y2}\ny3 = {y3}\nx o (y1,y2,y3) = {x.compose(y1,y2,y3)}')

#### Cyclic 

There are two important elements in $R[C_n]$

\begin{equation*}
T = \rho-1 \quad \text{ and } \quad N = 1+\rho+\cdots+\rho^{n-1}
\end{equation*}

defining a minimal resolution $W$

\begin{equation*}
R[C_n] \stackrel{T}{\longleftarrow} R[C_n] \stackrel{N}{\longleftarrow} R[C_n] \stackrel{T}{\longleftarrow} \cdots
\end{equation*}

of $R$ by a free differential graded $R[C_n]$-module.

**Implementation**. The class `CyclicModule_element` contains the methods `transposition_element` and `norm_element` returning models for $T$ and $N$. 

#### The map $\psi$ <a name="psi"></a>

**Definition**. The quasi-isomorphism $\psi : W \to N_\bullet(C_p;\mathbb Z/p\mathbb Z)$ is defined by

\begin{equation*}
\psi(e_{d}) = \begin{cases}
\displaystyle{\sum_{r_1, \dots, r_n}} \big(\rho^0, \rho^{r_1}, \rho^{r_1+1}, \dots, \rho^{r_{n}}, \rho^{r_{n}+1} \big) & d = 2n, \\
\displaystyle{\sum_{r_1, \dots, r_n}} \big(\rho^0, \rho^1, \rho^{r_1}, \rho^{r_1+1}, \dots, \rho^{r_{n}}, \rho^{r_{n}+1} \big) & d = 2n+1
\end{cases}
\end{equation*}

where the sum is over all $r_1, \dots, r_n \in \{1, \dots, p\}$.

**Implementation**. The class `CyclicModule_element` contains a method `psi` modeling this map.

In [ ]:
m = n = 3
N = CyclicModule_element.norm_element(torsion=m, order=n)

print(f'Let us consider the norm element for Z/{n}Z[C_{n}]\n'+
      f'in degree 2: N = {N} then \n'+
      f'psi(N) = {N.psi(2)}')

## Operads <a name="operads"></a>

**Definition**. Let $R$ be a ring. An operad $\mathcal O$ is a set $\big\{ \mathcal O(r) \big\}_{r \in \mathbb N}$ of differential graded left $R[\Sigma_r]$-modules and morphisms, called total composition,

\begin{equation*}
\circ_{\mathcal O} : \mathcal O(r) \otimes \mathcal O(s_1) \otimes \cdots \otimes \mathcal O(s_r) \to \mathcal O(s_1+\cdots+s_r)
\end{equation*}

satisfying: associativity, unitality and equivariance. 

Given integers $r > 0$, $s \geq 0$ and $k \in \{1, \dots, r\}$, the partial composition

\begin{equation*}
\circ_{k} : \mathcal O(r) \otimes \mathcal O(s) \to \mathcal O(r + s - 1)
\end{equation*}

is defined by 
\begin{equation*}
\circ_{k}(\tau, \sigma) = \circ_{\mathcal O}(\tau, 1, \dots, \sigma, \dots, 1)
\end{equation*}

where $\sigma$ is placed in $k$-th position among the copies of the unit $1 \in \mathcal O(1)$. 

We say $\mathcal O$ is an $E_\infty$-operad if $O(r)$ is a free $R[\Sigma_r]$-module resolution of $R$.

**Definition**. An algebra $A$, respectively a coalgebra $C$, over an operad $\mathcal O$ is a collection of maps

\begin{equation*}
\mathcal O(r) \otimes A^{\otimes r} \to A
\end{equation*}
respectively
\begin{equation*}
\mathcal O(r) \otimes C \to C^{\otimes r}
\end{equation*}

compatible with the operadic structure. 

These structures are respectively equivalent to operad maps

\begin{equation*}
\mathcal O \to \big\{ \mathrm{End}(A^{\otimes r}, A) \big\}_{r \geq 0}
\end{equation*}
and
\begin{equation*}
\mathcal O \to \big\{ \mathrm{End}(C, C^{\otimes r}) \big\}_{r \geq 0}
\end{equation*}

where the operad structure in the target is induced from the usual composition of linear maps, the group action is induced from permutations of factors, and the composition identity is the identity map.

### Barratt-Eccles operad $\mathcal E$ <a name="barrat_eccles"></a>

**Definition**. Let $R$ be a ring. The arity $r$ part $\mathcal E(r)$ of the Barratt-Eccles operad $\mathcal E$ is the coskeletal differential graded $R[\Sigma_r]$-module generated by $\Sigma_r$. Its composition is best described using Remark 1. 

The chain complex $\mathcal E(r)$ equals the normalized chains $N_\bullet$ of the simplicial set 

\begin{equation*}
\begin{split}
E(r)_n &= \{ (\sigma_0, \dots, \sigma_n)\ |\ \sigma_i \in \Sigma_r\}, \\
d_i(\sigma_0, \dots, \sigma_n) &= (\sigma_0, \dots, \widehat{\sigma}_i, \dots, \sigma_n), \\
s_i(\sigma_0, \dots, \sigma_n) &= (\sigma_0, \dots, \sigma_i, \sigma_i, \dots, \sigma_n). \\
\end{split}
\end{equation*}
with $\Sigma_r$-action given by 
\begin{equation*}
\sigma (\sigma_0, \dots, \sigma_n) = (\sigma\sigma_0, \dots, \sigma\sigma_n).
\end{equation*} 
Let
\begin{equation*}
{\circ}_{E}: E(r) \times E(s_1) \times \cdots \times E(s_r) \to E(s_1 + \cdots + s_r).
\end{equation*}

be defined by applying coordinatewise the composition of permutations.

We define the composition of $\mathcal E$ by precomposing $N_\bullet(\circ_E)$ with the iterated Eilenberg-Zilber map

\begin{equation*}
N_\bullet(E(r)) \otimes N_\bullet(E(s_1)) \otimes \cdots \otimes N_\bullet(E(s_r))
\longrightarrow
N_\bullet(E(r) \times E(s_1) \times \cdots \times E(s_r))
\end{equation*}


**Implementation**. We introduce a subclass of `DGModule_element` that models elements in the Barrat-Eccles operad when $R = \mathbb Z$ or $R = \mathbb Z/n\mathbb Z$. It inherents addition `+`, substraction `-`, scaling `*`, and `boundary` methods, additionally, it supports the left action of the correponding symmetric group ring `*`, and it contains the method `compose` modeling the operadic composition.

**Symmetric action**

In [ ]:
from clesto import BarrattEccles_element


b = BarrattEccles_element({((1,2,3),(3,2,1)):1})
x = SymmetricModule_element({(1,3,2):1})

print(f'Let b = {b} and x = {x} then \n' + 
      f'x * x = {x * b} \n' +
      f'b * x = {b * x}')

**Partial composition**

In [ ]:
b1 = BarrattEccles_element({((1,2),(2,1)):1, ((2,1),(1,2)):2})
b2 = BarrattEccles_element({((2,1,3),(2,3,1)):3})

print(f'b1 = {b1}\nb2 = {b2}\nb1 o_2 b2 = {b1.compose(b2, 2)}')

**Composition**

In [ ]:
b = BarrattEccles_element({((1,2,3),(1,3,2)):1})
c1 = BarrattEccles_element({((2,1),(1,2)):1})
c2 = BarrattEccles_element({((1,2,3),):2})
c3 = BarrattEccles_element({((2,1),):3})

print(f'For b  = {b} \n' +
      f'c1 = {c1} \n' +
      f'c2 = {c2} \n' + 
      f'c3 = {c3} \nwe have\n' +
      f'b o (c1,c2,c3) = \n{b.compose(c1,c2,c3)}')

#### The map $\phi$ <a name="phi"></a>

**Definition**. The generator $\rho$ of $C_p$ can be regaded as the permutation

\begin{equation*}
\rho(1,2,\dots,p) = (2,\dots,p,1)
\end{equation*}

which induces an inclusion of associated coskeletal differential graded modules

\begin{equation*}
\phi: N_\bullet(C_p;\mathbb Z/p\mathbb Z) \to N_\bullet(\Sigma_p;\mathbb Z/p\mathbb Z) = \mathcal E(p)
\end{equation*}

**Implementation**. The class `CyclicDGModule_element` contains a method `phi` modeling this map.

In [ ]:
x = CyclicDGModule_element({(0,5,0,5):1}, torsion=4, order=4)

print(f'Let x = {x} then \nphi(x) = {x.phi()}')

### Surjection operad <a name="surjection"></a>

**Definition** Let $R$ be a ring. The arity $r$ part $\mathcal Surj(r)$ of the Surjection operad $Surj$ is the coskeletal differential graded $R$-module generated by $\{1,\dots,r\}$ modulo tuples defining functions $\{1,\dots,r+d\} \to \{1,\dots,r\}$ that fail to be surjective. The symmetric group action is induced for any $\sigma \in \Sigma_r$ by

\begin{equation*}
\sigma (s(1), \dots, s(r+s)) = (\sigma s(1), \dots, \sigma s(r+s))
\end{equation*}

**Implementation**. We introduce a subclass of `DGModule_element` that models elements in the Surjection operad when $R = \mathbb Z$ or $R = \mathbb Z/n\mathbb Z$. It inherents addition `+`, substraction `-`, scaling `*`, and `boundary` method, additionally, it supports the left action of the correponding symmetric group ring `*`, and it contains the method `compose` modeling operadic composition.

In [ ]:
from clesto import Surjection_element

s = Surjection_element({(1,2,3,1):1, (2,3,2,1):2, (2,3,2,3):1})
x = SymmetricModule_element({(2,1,3):2})
print(f'Let s = {s} \nand x = {x} then \nx * s = {x * s}')

#### Composition (TO DO: description)

For all integers $r, s, d, e \geq 1$ and every $k \in \{1, \ldots, s\}$ we define a partial composition

\begin{equation*}
\circ_k : Surj(r)_d \otimes Surj(s)_e \to \mathcal Surj(r + s - 1)_{d+e} \, .
\end{equation*}

In [ ]:
s1 = Surjection_element({(1, 2, 1, 3): 1})
s2 = Surjection_element({(1, 2, 1): 1})

print(f's1 = {s1}\ns2 = {s2}\ns1 o_2 s2 = {s1.compose(s2, 1)}')

We also define a composition
\begin{equation*}
\circ : Surj(r)_d \otimes Surj(s_1)_d \otimes \cdots \otimes Surj(s_r)_d \to Surj(s_1 + \cdots + s_r)_{(r+1)d} \, .
\end{equation*}

In [ ]:
s = Surjection_element({(1, 2, 1, 3): 1})
t1 = Surjection_element({(1, 2): 1})
t2 = Surjection_element({(2, 1): 1})
t3 = Surjection_element({(1, 2): 1})

print(f'For s  = {s} \n' +
      f't1 = {t1} \n' +
      f't2 = {t2} \n' + 
      f't3 = {t3} \nwe have\n' +
      f's o (t1,t2,t3) = \n{s.compose(t1,t2,t3)}')

#### Table reduction morphism <a name="table_reduction"></a>

**Definition**. The table reduction morphism 
\begin{equation*}
TR : \mathcal E \to Surj
\end{equation*}
is a quasi-ismomorphism introduced in [[9]](#9) that we now review.

Given a basis element $(\sigma_0, \dots, \sigma_n) \in \mathcal E(r)_n$ we define
\begin{equation*}
TR(\sigma_0, \dots, \sigma_n) = \sum_{a} s_{a}
\end{equation*}
as a sum of surjections
\begin{equation*}
s_{a} : \{1, \dots, n+r \} \to \{1, \dots, r\}
\end{equation*}

parametrized by all $a = (a_0, \dots, a_n)$ with each $a_i \geq 1$ and $a_0 + \cdots + a_n = n + r$.

For one such tuple $a$ we now describe its associated surjection $s_a$. Define recursively

\begin{equation*}
A_{-1} = 0 \qquad A_i = A_{i-1} + a_{i.}
\end{equation*}

For $k \in \{1, \dots, n+r\}$ we identify $i \in \{1, \dots, n\}$ such that $A_{i-1} < k \leq A_{i}$ and define $s_a(k)$ to be the $(k - A_{i-1})$-th element in $(\sigma_i(1), \dots, \sigma_i(r))$ not in

\begin{equation*}
\big\{ s_a(j) \ | \ j < k \text{ and } j \neq A_0, \dots , A_{i-1} \big\}.
\end{equation*}
    
**Implementation**. The class `BarrattEccles_element` contains the method `table_reduction` modeling this morphism.

In [ ]:
b = BarrattEccles_element({((1,2,3),(1,3,2)):1})
print(f'If b = {b} then TR(b) = {b.table_reduction()}')

### Eilenberg-Zilber operad <a name="eilenberg_zilber"></a>

**Definition**. A simplicial set $X = \{X_n\}_{n \geq 0}$ is a cellection of sets together with maps

\begin{equation*}
d_i \colon X_{n} \to X_{n-1}
\qquad 
s_i \colon X_{n} \to X_{n+1}
\end{equation*}

for $i = 0, \dots, n$ satisfying the so called simplicial identities.

The functor $N_\bullet$ from simplicial sets to chain complexes is defined by

\begin{equation*}
N_n(X; R) = \frac{R\{X_n\}}{R\{s(X_{n-1})\}} \qquad \partial = \sum (-1)^i d_i
\end{equation*}

where $s(X_{n-1}) = \bigcup_{i=0}^n s_i(x_n)$, the so called subset of degenerate simplices, and $d_i$ has been $R$-linearly extended. For any $U = \{u_1 < \cdots < u_m\}$ where $u_i \geq 0$ we use the notation

\begin{equation*}
d_U = d_{u_1} \dots d_{u_{m.}}
\end{equation*}

**Definition**. The Eilenberg-Zilber operad $\mathrm{End}(N_\bullet)$ has arity $r$ given by all $R$-linear natural transformations

\begin{equation*}
N_\bullet \to N_\bullet^{\otimes r}
\end{equation*}

with operad structure induced from the one on $\big\{\mathrm{Hom}(C, C^{\otimes r})\big\}_{r \geq 0}$ for any chain complex $C$.

More explicitly, an element in arity $r$ of degree $d$ is a set $op = \{op_n\}_{n \geq 0}$ with

\begin{equation*}
op_n = \sum_i r_i d_{U^i_1} \otimes \cdots \otimes d_{U^i_r}
\end{equation*}
such that: 
1. $\forall i,j \ \ U^i_j \subseteq \{0,\dots,n\}$,
2. $\forall i \ \bigcap_{j=1}^r U^i_j = \emptyset$,
3. $\forall i \ \sum_{j=1}^r |U^i_j| = r(n-1)-d$.

Any simplicial set defines a coalgebra over the Eilenberg-Zilber operad given by 

\begin{equation*}
op_n(x)= \sum_i r_i d_{U^i_1}(x) \otimes \cdots \otimes d_{U^i_r}(x).
\end{equation*}
for $x \in X_n$ and $op \in \mathrm{End}(N_\bullet)$.

**Implementation**. We introduce a subclass of `Module_element` that models elements in the Eilenberg-Zilber operad when $R = \mathbb Z$ or $R = \mathbb Z/n\mathbb Z$. It inherents addition `+`, substraction `-`, scaling `*` and, additionally, it supports the left action of the correponding symmetric group ring `*`, contains the method `compose` modeling operadic composition and carries the attribute `arity`.

In [ ]:
from clesto import EilenbergZilber_element

x = EilenbergZilber_element({((1,2), (1,2), tuple()):2,
                             ((1,2), (1,1,2), (1,)):3})

print(f'If x = {x} \nthen the arity of x is {x.arity}')

**Definition** The standard infinite simplex $\Delta^\infty$ is the simplicial set with non-degenerate $n$-simplices the subsets of $\mathbb N$ of cardinality $n+1$ denoted $[v_0, \dots, v_n]$ with $v_i < v_j$ if $i < j$, and 
\begin{align*}
d_i[v_0, \dots, v_n] & = [v_0, \dots, \widehat{v}_i, \dots, v_n] \\
s_i[v_0, \dots, v_n] & = [v_0, \dots, v_i, v_i, \dots, v_n].
\end{align*}

Let $op = \{op_n\}_{n \geq 0}$ be an eement in $\mathrm{End}(N_\bullet)$. Using the representability of simplicial sets, the element $op_n$ is determined by its action on the basis element $[0,\dots,n] \in N_{n}(\Delta^\infty; R)$.

**Implementation**. The class `EilenbergZilber_element` contains the method `__act__` that allows us to think of it as a funtion taking as input an `int`, say $n$, and returning a model of $op_n([0, \dots, n])$.

In [ ]:
op = EilenbergZilber_element({((1,2), (0,3)):1, ((1,), ()):2})
n = 3
print(f'Let op = {op} then \n' + 
      f'op({tuple(range(n+1))}) = {op(n)}')

#### Interval cut morphism <a name="interval_cut"></a>

**Definition**. The interval cut morphism
\begin{equation*}
IC : Surj \to \mathrm{End}(N_\bullet)
\end{equation*}
is a quasi-isomorphism described in [[9]](#9)

DESCRIBE

**Implementation**. The class `Surjection_element` has the method `interval_cut` that models this map. 

In [ ]:
n = 2
surj = Surjection_element({(3,1,2,3,2):1}, torsion='free')
print(f'If surj = {surj} then its action on {n}-simplices is given by \n' +
      f'IC(surj) = {surj.interval_cut(n)}')

## Steenrod operations <a name="steenrod_operations"></a>

**Convention**. For the rest of this notebook we work over $\mathbb Z/ p \mathbb Z$ where $p$ is a prime number.

### Chain convention <a name="chain_convention"></a>

**Definition**. Let $\mathcal O$ be an $E_\infty$-operad,

\begin{equation*}
\iota : W \to \mathcal O(p)
\end{equation*}

a choice of $\mathbb F_p[\mathrm C_p]$-equivariant quasi-isomorphism and $A$ and $\mathcal O$-algebra. The map

\begin{equation*}
D_d : A \to A
\end{equation*}
is defined by
\begin{equation*}
D_d(a) = \begin{cases}
\iota(e_d)(a^{\otimes p})& d \geq 0 \\
0 & d < 0.
\end{cases}
\end{equation*}
For any integer $s$, the Steenrod operations

\begin{equation*}
P_s : H_\bullet(A) \to H_{\bullet + s}(A)
\end{equation*}
for $p = 2$ and
\begin{equation*}
P_s : H_\bullet(A) \to H_{\bullet + 2s(p-1)}(A) \qquad
\beta P_s : H_\bullet(A) \to H_{\bullet + 2s(p-1) - 1}(A)
\end{equation*}

for $p > 2$, are defined for a class $[a]$ of degree $n$ respectively by

\begin{equation*}
P_s\big([a]\big) = \big[D_{s-n}(a)\big] \qquad
\end{equation*}
and
\begin{equation*}
P_s\big([a]\big) = \big[(-1)^s \nu(n) D_{(2s-n)(p-1)}(a)\big] \qquad
\beta P_s\big([a]\big) = \big[(-1)^s \nu(n)D_{(2s-n)(p-1)-1}(a)\big]
\end{equation*}

where $\nu(n) = (-1)^{n(n-1)m/2}(m!)^n$ and $m = (p-1)/2$.

**Remark 2**. The use of the indices $(2s-n)(p-1)$ and $(2s-n)(p-1)-1$ in the definition of odd Steenrod operations is motivated by a result of Thom, stating that other indices give rise to operations that are identically $0$.

**Remark 3**. The use of the coefficient function $\nu(n)$ is credited to Serre and motivated by the identity $D_{n(p-1)}(a) = \nu(n)a$ for $a$ of degree $n$.

**Remark 4**. The notation $\beta P_s$ is motivated by the relationship of this operator and the Bockstein of the reduction $\mathbb Z \to \mathbb Z/p\mathbb Z$.

**Implementation**: We introduce a class modeling Steenrod operations at the chain level. More precisely, `SteenrodOperation(p, s, n)` models $P^s$ acting on degree $n$ elements over the field with $p$ elements. The two methods `as_BarrattEccles_element` and `as_Surjection_element` are such that their outputs, say $x$ and $y$ respectively, satisfy 
\begin{equation*}
[x(a^{\otimes p})] = P_s([a]) \qquad
[y(a^{\otimes p})] = P_s([a])
\end{equation*}

for any $n$-cycle $a$ in an algebra over $\mathcal E$ or $Surj$ respectively.

When the optional boolean argument `bockstein` is `True`, the instance `PowerOperation(p, s, n, bockstein=True)` models $\beta P^s$.

In [ ]:
from clesto import SteenrodOperation

for p, s, n, b in ((2, 3, 2, False), (3, 2, 3, True)):
    op = SteenrodOperation(p, s, n, bockstein=b)
    print(f'*) Over Z/{p}Z, the operation {str(op)} is represented \n' +
          f'in the Barratt-Eccles operad by \nx = {op.as_BarrattEccles_element()} \n' +
          f'and in the Surjection operad by \ny = {op.as_Surjection_element()}.')

### Cochain convention <a name="cochain_convention"></a>

Let us consider now an algebra $A = A^\bullet$ over an $E_\infty$-operad whose differential increases degree. We can regrade so that $A^n = A_{-n}$ and define the Steenrod operations

\begin{equation*}
P^s : H^\bullet(A) \to H^{\bullet + s}(A)
\end{equation*}
for $p = 2$ and
\begin{equation*}
P^s : H^\bullet(A) \to H^{\bullet + 2s(p-1)}(A) \qquad
\beta P^s : H^\bullet(A) \to H^{\bullet + 2s(p-1) + 1}(A)
\end{equation*}

for $p > 2$ by $P^s = P_{-s}$ and $\beta P^s = \beta P_{-s}$.

**Remark**. It is customary to write $Sq^s$ when $p = 2$ instead of $P^s$.

**Implementation**. The `SteenrodOperation` class has an optional argument `convention` whose default is `'chain'`. An instance `SteenrodOperation(p, s, n, convention='cochain', bockstein=b)` models either $P^s$ or $\beta P^s$ depending on `b`.

In [ ]:
for p, s, n, b in ((2, 2, 3, False), (3, 2, 5, False)):
    op = SteenrodOperation(p, s, n, bockstein=b, convention='cochain')
    print(f'*) Over Z/{p}Z, the operation {str(op)} is represented \n' +
          f'in the Barratt-Eccles operad by \nx = {op.as_BarrattEccles_element()} \n' +
          f'and in the Surjection operad by \ny = {op.as_Surjection_element()}.')

## Cochains of spaces <a name="spaces"></a>

**Definition**. The chains of any simplicial set define a coalgebra over the Eilenberg-Zilber operad and, by linear duality, its cochains define an algebra. The predual representative of a Power operation element is the Eilenberg-Zilber element $op = \{op_n\}$ such that

\begin{equation*}
P^s\big([\alpha]\big) = \big[ (\alpha \otimes \cdots \otimes \alpha) op_n(-)\big]
\end{equation*}

for any cocycle $\alpha$ of degree $n$.

Predual representatives generalize Steenrod's cup-$i$ coproducts, which are fundamentally rooted in the combinatorics of simplices [[10, 11]](#1011) and are playing an interesting role in condensed matter physics [[12, 13]](#1213).

**Implementation**. The `PowerOperation` class has the method `as_EilenbergZilber_element` returning the predual representative of the power operation.

In [ ]:
for p, s, n, b in ((2, 2, 2, False), (3, 1, 2, False)):
    # degree of P^s(x) w/ |x| = n
    if p == 2:
        m = n + s 
    elif p > 2:
        m = n + 2 * s * (p-1) + int(b)
        
    op = SteenrodOperation(p, s, n, bockstein=b, convention='cochain')  
    ez = op.as_EilenbergZilber_element()
    
    # filter out assuming a is a homogenous cochain
    homogeneous_ez = EilenbergZilber_element({k: v for k, v in ez.items()
        if len(set((len(x) for x in k))) == 1})
    
    print(f'*) Over Z/{p}Z, the action of the cocycle representing {op}(x)\n'
          f'on {tuple(range(m+1))} with x a homogeneous cocycle is \n' +
          f'{("x",)*p}{homogeneous_ez(m)}')

In [ ]:
p,s,q,b = 2, 2, 5, False
op = SteenrodOperation(p, s, q, bockstein=b, convention='cochain')
ez = op.as_EilenbergZilber_element()
homogeneous_ez = EilenbergZilber_element({k: v for k, v in ez.items()
    if len(set((len(x) for x in k))) == 1})

print(f'*) {op}(x) with |x| = {q} is \n' +
  f'{homogeneous_ez(s+q)}')

## References <a name="references"></a>

[1]<a name="1"></a> Steenrod, Norman E. "Products of cocycles and extensions of mappings." Annals of Mathematics (1947): 290-320.

[2]<a name="23"></a> Steenrod, N. E. "Homology groups of symmetric groups and reduced power operations." Proceedings of the National Academy of Sciences of the United States of America 39.3 (1953): 213.

[3] Steenrod, N. E. "Cyclic reduced powers of cohomology classes." Proceedings of the National Academy of Sciences of the United States of America 39.3 (1953): 217.

[4]<a name="45"></a> Kudo, Tatsuji, and Shôrô Araki. "Topology of $H_n$-spaces and $H$-squaring operations." Memoirs of the Faculty of Science, Kyushu University. Series A, Mathematics 10.2 (1956): 85-120.

[5] Dyer, Eldon, and Richard K. Lashof. "Homology of iterated loop spaces." American Journal of Mathematics 84.1 (1962): 35-88.

[6]<a name="6"></a> May, J. Peter. "A general algebraic approach to Steenrod operations." The Steenrod Algebra and its Applications: a conference to celebrate NE Steenrod's sixtieth birthday. Springer, Berlin, Heidelberg, 1970.

[7]<a name="7"></a> May, J. Peter. The geometry of iterated loop spaces. Vol. 271. Springer, 2006.

[8]<a name="89"></a> McClure, James, and Jeffrey Smith. "Multivariable cochain operations and little 𝑛-cubes." Journal of the American Mathematical Society 16.3 (2003): 681-704.

[9] Berger, Clemens, and Benoit Fresse. "Combinatorial operad actions on cochains." Mathematical Proceedings of the Cambridge Philosophical Society. Vol. 137. No. 1. Cambridge University Press, 2004.

[10]<a name="1011"></a> Medina-Mardones, Anibal M. "An axiomatic characterization of Steenrod's cup-$ i $ Products." arXiv preprint arXiv:1810.06505 (2018).

[11] Medina-Mardones, Anibal M. "An algebraic representation of globular sets." arXiv preprint arXiv:1906.01011 (2019).

[12]<a name="1213"></a> Kapustin, Anton, and Ryan Thorngren. "Fermionic SPT phases in higher dimensions and bosonization." Journal of High Energy Physics 2017.10 (2017): 80.

[13] Brumfiel, Greg, and John Morgan. "The pontrjagin dual of 4-dimensional spin bordism." arXiv preprint arXiv:1803.08147 (2018).